In [2]:
#import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



### deribit API Request

In [3]:
""" Download CC-Option Data from Deribit via public API """

"""
Matteo Bottacini -- matteo.bottacini@usi.ch
"""

# import modules
import json
import requests
from tqdm import tqdm
import sqlite3
import datetime


# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin
    """

    # get option name and settlement
    coin_name = get_option_name_and_settlement(coin)[0]
    settlement_period = get_option_name_and_settlement(coin)[1]

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df




In [4]:
# print data and time for log
print('Date and time: ' +  datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' , format: dd/mm/yyyy hh:mm:ss')

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')
eth_data = get_option_data('ETH')

# The data is now stored in Pandas DataFrames 'btc_data' and 'eth_data'
print('BTC data collected')
print(btc_data.head()) # Display first few rows of BTC data
print('ETH data collected')
print(eth_data.head()) # Display first few rows of ETH data

Date and time: 03/02/2024 09:09:29 , format: dd/mm/yyyy hh:mm:ss


100%|██████████| 592/592 [03:38<00:00,  2.71it/s]

BTC data collected
   best_bid_amount  best_ask_amount  bid_iv  ask_iv underlying_index  \
0              0.0             20.7     0.0  151.10   SYN.BTC-4FEB24   
0              0.0             92.5     0.0  110.01   SYN.BTC-4FEB24   
0              0.0             19.8     0.0  139.25   SYN.BTC-4FEB24   
0              0.0              1.5     0.0   90.44   SYN.BTC-4FEB24   
0              0.0             20.7     0.0  126.92   SYN.BTC-4FEB24   

   underlying_price  mark_iv  interest_rate  best_bid_price  best_ask_price  \
0        43046.8715    85.49            0.0             0.0          0.1070   
0        43046.6290    85.49            0.0             0.0          0.0002   
0        43046.8715    78.77            0.0             0.0          0.0955   
0        43046.8715    78.77            0.0             0.0          0.0001   
0        43046.8715    66.52            0.0             0.0          0.0840   

   ...  greeks.theta  greeks.vega  greeks.gamma greeks.delta stats.volume

In [5]:
btc_data.to_csv('data/btc_data.csv', index=False)
eth_data.to_csv('data/eth_data.csv', index=False)

In [6]:
btc_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,20.7,0.0,151.10,SYN.BTC-4FEB24,43046.8715,85.49,0.0,0.0,0.1070,...,-4.77329,0.11167,0.00000,0.9982,0.00,0.0,None,None,None,day
0,0.0,92.5,0.0,110.01,SYN.BTC-4FEB24,43046.6290,85.49,0.0,0.0,0.0002,...,-0.87260,0.11172,0.00000,-0.0018,0.00,0.0,None,None,None,day
0,0.0,19.8,0.0,139.25,SYN.BTC-4FEB24,43046.8715,78.77,0.0,0.0,0.0955,...,-6.15599,0.15630,0.00001,0.9974,0.00,0.0,None,None,None,day
0,0.0,1.5,0.0,90.44,SYN.BTC-4FEB24,43046.8715,78.77,0.0,0.0,0.0001,...,-1.19951,0.15630,0.00001,-0.0026,12.92,3.0,0.0,0.0001,0.0001,day
0,0.0,20.7,0.0,126.92,SYN.BTC-4FEB24,43046.8715,66.52,0.0,0.0,0.0840,...,-4.09321,0.12306,0.00000,0.9980,0.00,0.0,None,None,None,day


In [7]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658 entries, 0 to 0
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     658 non-null    float64
 1   best_ask_amount     658 non-null    float64
 2   bid_iv              658 non-null    float64
 3   ask_iv              658 non-null    float64
 4   underlying_index    658 non-null    object 
 5   underlying_price    658 non-null    float64
 6   mark_iv             658 non-null    float64
 7   interest_rate       658 non-null    float64
 8   best_bid_price      658 non-null    float64
 9   best_ask_price      658 non-null    float64
 10  open_interest       658 non-null    float64
 11  max_price           658 non-null    float64
 12  min_price           658 non-null    float64
 13  last_price          501 non-null    object 
 14  asks                658 non-null    object 
 15  bids                658 non-null    object 
 16  settlement

In [8]:
eth_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,198.0,0.0,134.69,SYN.ETH-4FEB24,2303.6341,52.50,0.0,0.0000,0.0900,...,-0.00492,0.00019,0.00000,0.99996,0.0,0.0,None,None,None,day
0,0.0,180.0,0.0,99.24,SYN.ETH-4FEB24,2303.6341,52.50,0.0,0.0000,0.0003,...,-0.00055,0.00019,0.00000,-0.00004,0.0,0.0,None,None,None,day
0,0.0,209.0,0.0,110.36,SYN.ETH-4FEB24,2303.6341,43.13,0.0,0.0000,0.0685,...,-0.01572,0.00073,0.00002,0.99982,0.0,0.0,None,None,None,day
0,0.0,720.0,0.0,81.04,SYN.ETH-4FEB24,2303.6341,43.13,0.0,0.0000,0.0004,...,-0.00206,0.00073,0.00002,-0.00018,0.0,0.0,None,None,None,day
0,3.0,166.0,0.0,89.76,SYN.ETH-4FEB24,2303.6341,43.00,0.0,0.0265,0.0475,...,-0.51708,0.02405,0.00052,0.99148,0.0,0.0,None,None,None,day


In [9]:
eth_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 0
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     592 non-null    float64
 1   best_ask_amount     592 non-null    float64
 2   bid_iv              592 non-null    float64
 3   ask_iv              592 non-null    float64
 4   underlying_index    592 non-null    object 
 5   underlying_price    592 non-null    float64
 6   mark_iv             592 non-null    float64
 7   interest_rate       592 non-null    float64
 8   best_bid_price      592 non-null    float64
 9   best_ask_price      592 non-null    float64
 10  open_interest       592 non-null    float64
 11  max_price           592 non-null    float64
 12  min_price           592 non-null    float64
 13  last_price          437 non-null    object 
 14  asks                592 non-null    object 
 15  bids                592 non-null    object 
 16  settlement